# Pruebas con tests estadísticos para comprobar el salto térmico

In [1]:
import numpy as np
import pandas as pd
import pickle
import scipy.stats as stats
from scipy.stats import ttest_rel, shapiro, wilcoxon, t
import matplotlib.pyplot as plt

In [2]:
fecha_ref = pd.Timestamp('2025-01-01 00:00:00').floor('d')

In [ ]:
# Definir rangos de días para probar
dias_antes = [-3, -2, -1, 0] 
dias_despues = [2, 3, 4, 5, 6] 

### Series alineadas

In [4]:
with open("../procData/muestras_ovul_horas_norm1.pkl", "rb") as f:
    muestras_alin = pickle.load(f)

In [5]:
resultados_alin = []

for dia_antes in dias_antes:
    for dia_despues in dias_despues:
        valores_antes = []
        valores_despues = []

        for key, data in muestras_alin.items():
            df = data["serie"].copy()
            # Calcular días relativos (decimales)
            df["dias_rel"] = (df["resultTimestamp"] - fecha_ref) / pd.Timedelta(days=1)
            
            # Agrupar por día entero redondeado para obtener medias diarias
            df["dia_entero"] = df["dias_rel"].round().astype(int)
            medias_diarias = df.groupby("dia_entero")["result"].mean()

            # Extraer medias del día antes y día después
            # Si no hay datos para ese día, se puede usar NaN o saltar ese sujeto
            val_antes = medias_diarias.get(dia_antes, np.nan)
            val_despues = medias_diarias.get(dia_despues, np.nan)

            # Sólo añadimos si ambos valores existen
            if not (np.isnan(val_antes) or np.isnan(val_despues)):
                valores_antes.append(val_antes)
                valores_despues.append(val_despues)

        valores_antes = np.array(valores_antes)
        valores_despues = np.array(valores_despues)
        diferencias = valores_despues - valores_antes

        # Wilcoxon test para ascenso (antes < despues)
        stat_wil, p_wil = wilcoxon(valores_antes, valores_despues, alternative='less')

        # Shapiro para normalidad de diferencias
        stat_shap, p_shap = shapiro(diferencias)

        # Decide qué test t hacer o no
        if p_shap > 0.05:
            # Normalidad aceptada -> test t pareado
            stat_t, p_t = ttest_rel(valores_despues, valores_antes)
        else:
            stat_t, p_t = np.nan, np.nan  # no se hace test t

        # Media e intervalo confianza diferencia
        media_dif = np.mean(diferencias)
        std_dif = np.std(diferencias, ddof=1)
        n = len(diferencias)

        alpha = 0.05
        t_crit = t.ppf(1 - alpha/2, df=n-1)
        error_estandar = std_dif / np.sqrt(n)
        ic_inf = media_dif - t_crit * error_estandar
        ic_sup = media_dif + t_crit * error_estandar

        resultados_alin.append({
            "dia_antes": dia_antes,
            "dia_despues": dia_despues,
            "wilcoxon_p": p_wil,
            "shapiro_p": p_shap,
            "ttest_p": p_t,
            "media_dif": media_dif,
            "std_err": error_estandar,
            "int_conf_inf": ic_inf,
            "int_conf_sup": ic_sup
        })

df_resultados_alin = pd.DataFrame(resultados_alin)

print("Top resultados ordenados por error estándar en la diferencia:")
print(df_resultados_alin.sort_values("wilcoxon_p").head(10)[
    ["dia_antes", "dia_despues", "wilcoxon_p", "shapiro_p", "ttest_p", "media_dif", "std_err", "int_conf_inf", "int_conf_sup"]
])


Top resultados ordenados por error estándar en la diferencia:
    dia_antes  dia_despues  wilcoxon_p  shapiro_p   ttest_p  media_dif  \
11         -1            3    0.000006   0.001174       NaN   0.061000   
6          -2            3    0.000008   0.000542       NaN   0.062136   
12         -1            4    0.000009   0.029508       NaN   0.056828   
1          -3            3    0.000010   0.007244       NaN   0.062798   
2          -3            4    0.000041   0.201102  0.000227   0.058627   
7          -2            4    0.000051   0.071249  0.000491   0.057964   
3          -3            5    0.000065   0.104604  0.000229   0.063001   
8          -2            5    0.000076   0.243531  0.000461   0.062338   
5          -2            2    0.000099   0.005295       NaN   0.046112   
10         -1            2    0.000112   0.467437  0.000303   0.044977   

     std_err  int_conf_inf  int_conf_sup  
11  0.014360      0.032255      0.089745  
6   0.015036      0.032038      0.092

### Series normalizadas

In [6]:
with open("../procData/muestras_ovul_horas_norm2.pkl", "rb") as f:
    muestras_norm = pickle.load(f)

In [7]:
resultados_norm = []

for dia_antes in dias_antes:
    for dia_despues in dias_despues:
        valores_antes = []
        valores_despues = []

        for key, data in muestras_norm.items():
            df = data["serie"].copy()
            # Calcular días relativos (decimales)
            df["dias_rel"] = (df["resultTimestamp"] - fecha_ref) / pd.Timedelta(days=1)
            
            # Agrupar por día entero redondeado para obtener medias diarias
            df["dia_entero"] = df["dias_rel"].round().astype(int)
            medias_diarias = df.groupby("dia_entero")["result"].mean()

            # Extraer medias del día antes y día después
            # Si no hay datos para ese día, se puede usar NaN o saltar ese sujeto
            val_antes = medias_diarias.get(dia_antes, np.nan)
            val_despues = medias_diarias.get(dia_despues, np.nan)

            # Sólo añadimos si ambos valores existen
            if not (np.isnan(val_antes) or np.isnan(val_despues)):
                valores_antes.append(val_antes)
                valores_despues.append(val_despues)

        valores_antes = np.array(valores_antes)
        valores_despues = np.array(valores_despues)
        diferencias = valores_despues - valores_antes

        # Wilcoxon test para ascenso (antes < despues)
        stat_wil, p_wil = wilcoxon(valores_antes, valores_despues, alternative='less')

        # Shapiro para normalidad de diferencias
        stat_shap, p_shap = shapiro(diferencias)

        # Decide qué test t hacer o no
        if p_shap > 0.05:
            # Normalidad aceptada -> test t pareado
            stat_t, p_t = ttest_rel(valores_despues, valores_antes)
        else:
            stat_t, p_t = np.nan, np.nan  # no se hace test t

        # Media e intervalo confianza diferencia
        media_dif = np.mean(diferencias)
        std_dif = np.std(diferencias, ddof=1)
        n = len(diferencias)

        alpha = 0.05
        t_crit = t.ppf(1 - alpha/2, df=n-1)
        error_estandar = std_dif / np.sqrt(n)
        ic_inf = media_dif - t_crit * error_estandar
        ic_sup = media_dif + t_crit * error_estandar

        resultados_norm.append({
            "dia_antes": dia_antes,
            "dia_despues": dia_despues,
            "wilcoxon_p": p_wil,
            "shapiro_p": p_shap,
            "ttest_p": p_t,
            "media_dif": media_dif,
            "std_err": error_estandar,
            "int_conf_inf": ic_inf,
            "int_conf_sup": ic_sup
        })

df_resultados_norm = pd.DataFrame(resultados_norm)

print("Top resultados ordenados por error estándar en la diferencia:")
print(df_resultados_norm.sort_values("wilcoxon_p").head(10)[
    ["dia_antes", "dia_despues", "wilcoxon_p", "shapiro_p", "ttest_p", "media_dif", "std_err", "int_conf_inf", "int_conf_sup"]
])


Top resultados ordenados por error estándar en la diferencia:
    dia_antes  dia_despues  wilcoxon_p  shapiro_p   ttest_p  media_dif  \
11         -1            3    0.000005   0.170149  0.000009   0.425357   
12         -1            4    0.000008   0.034137       NaN   0.358314   
6          -2            3    0.000008   0.024812       NaN   0.427624   
1          -3            3    0.000015   0.153971  0.000028   0.422401   
13         -1            5    0.000039   0.483251  0.000013   0.429586   
3          -3            5    0.000041   0.409952  0.000088   0.426630   
2          -3            4    0.000056   0.049897       NaN   0.355357   
8          -2            5    0.000065   0.884619  0.000091   0.431853   
10         -1            2    0.000073   0.440218  0.000113   0.325392   
7          -2            4    0.000091   0.170498  0.000319   0.360581   

     std_err  int_conf_inf  int_conf_sup  
11  0.087434      0.250338      0.600376  
12  0.077987      0.202206      0.514

### Tendencia de las series

In [8]:
with open("../procData/muestras_ovul_tend.pkl", "rb") as f:
    muestras_tendencia = pickle.load(f)

In [9]:
resultados = []

for dia_antes in dias_antes:
    for dia_despues in dias_despues:
        valores_antes = []
        valores_despues = []

        for key, data in muestras_tendencia.items():
            df = data["serie"].copy()
            # Calcular días relativos (decimales)
            df["dias_rel"] = (df["resultTimestamp"] - fecha_ref) / pd.Timedelta(days=1)
            
            # Agrupar por día entero redondeado para obtener medias diarias
            df["dia_entero"] = df["dias_rel"].round().astype(int)
            medias_diarias = df.groupby("dia_entero")["tendencia"].mean()

            # Extraer medias del día antes y día después
            # Si no hay datos para ese día, se puede usar NaN o saltar ese sujeto
            val_antes = medias_diarias.get(dia_antes, np.nan)
            val_despues = medias_diarias.get(dia_despues, np.nan)

            # Sólo añadimos si ambos valores existen
            if not (np.isnan(val_antes) or np.isnan(val_despues)):
                valores_antes.append(val_antes)
                valores_despues.append(val_despues)

        valores_antes = np.array(valores_antes)
        valores_despues = np.array(valores_despues)
        diferencias = valores_despues - valores_antes

        # Wilcoxon test para ascenso (antes < despues)
        stat_wil, p_wil = wilcoxon(valores_antes, valores_despues, alternative='less')

        # Shapiro para normalidad de diferencias
        stat_shap, p_shap = shapiro(diferencias)

        # Decide qué test t hacer o no
        if p_shap > 0.05:
            # Normalidad aceptada -> test t pareado
            stat_t, p_t = ttest_rel(valores_despues, valores_antes)
        else:
            stat_t, p_t = np.nan, np.nan  # no se hace test t

        # Media e intervalo confianza diferencia
        media_dif = np.mean(diferencias)
        std_dif = np.std(diferencias, ddof=1)
        n = len(diferencias)

        alpha = 0.05
        t_crit = t.ppf(1 - alpha/2, df=n-1)
        error_estandar = std_dif / np.sqrt(n)
        ic_inf = media_dif - t_crit * error_estandar
        ic_sup = media_dif + t_crit * error_estandar

        resultados.append({
            "dia_antes": dia_antes,
            "dia_despues": dia_despues,
            "wilcoxon_p": p_wil,
            "shapiro_p": p_shap,
            "ttest_p": p_t,
            "media_dif": media_dif,
            "std_err": error_estandar,
            "int_conf_inf": ic_inf,
            "int_conf_sup": ic_sup
        })

df_resultados = pd.DataFrame(resultados)

print("Top resultados ordenados por error estándar en la diferencia:")
print(df_resultados.sort_values("wilcoxon_p").head(10)[
    ["dia_antes", "dia_despues", "wilcoxon_p", "shapiro_p", "ttest_p", "media_dif", "std_err", "int_conf_inf", "int_conf_sup"]
])


Top resultados ordenados por error estándar en la diferencia:
    dia_antes  dia_despues  wilcoxon_p  shapiro_p   ttest_p  media_dif  \
6          -2            3    0.000003   0.060195  0.000013   0.054793   
11         -1            3    0.000004   0.007297       NaN   0.049024   
10         -1            2    0.000007   0.369774  0.000015   0.045103   
5          -2            2    0.000012   0.093179  0.000005   0.050872   
12         -1            4    0.000016   0.198937  0.000044   0.045422   
7          -2            4    0.000020   0.721362  0.000029   0.051190   
8          -2            5    0.000024   0.430456  0.000059   0.045211   
9          -2            6    0.000035   0.899090  0.000027   0.050512   
1          -3            3    0.000063   0.006695       NaN   0.047049   
14         -1            6    0.000119   0.828160  0.000259   0.044743   

     std_err  int_conf_inf  int_conf_sup  
6   0.011494      0.031785      0.077801  
11  0.011164      0.026678      0.071

### Valores óptimos

In [10]:
# Mejores valores:
dia_antes_opt = -1
dia_despues_opt = 3

In [12]:
RANGE_VISUALIZ_Y = [36.0, 37.0]
RANGE_VISUALIZ_X = [-5, 8]

def print_series(dia_antes_opt, dia_despues_opt):
    fecha_ref = pd.Timestamp('2025-01-01 00:00:00').floor('h')

    for id in muestras_tendencia.keys(): 
        serie = muestras_tendencia[id]["serie"]

        # Convertimos las fechas a días relativos a la ovulación
        x = (serie["resultTimestamp"] - fecha_ref) / pd.Timedelta(days=1)
        y = serie["tendencia"]

        plt.figure(figsize=(8, 4))

        plt.plot(x, y, label="Temperatura", linewidth=2.5, color='#2980B9')

        plt.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Ovulación')
        plt.axvline(x=dia_antes_opt, color='green', linestyle='--', linewidth=2, label=f'Día {dia_antes_opt}')
        plt.axvline(x=dia_despues_opt, color='orange', linestyle='--', linewidth=2, label=f'Día {dia_despues_opt}')

        plt.title(f'Serie {id}')
        plt.xlabel('Día relativo a la ovulación')
        plt.ylabel('Temperatura (°C)')
        plt.grid(True)

        plt.xlim(RANGE_VISUALIZ_X[0], RANGE_VISUALIZ_X[1])
        plt.ylim(RANGE_VISUALIZ_Y[0], RANGE_VISUALIZ_Y[1])  

        plt.legend()
        plt.tight_layout()
        plt.show()


In [ ]:
print_series(dia_antes_opt, dia_despues_opt)